In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchvision.transforms as tt
from torch.utils.data import DataLoader, ConcatDataset
import torchvision

stats = ((0.2860,), (0.3530,))
batch_size = 1024


def get_transformations():
    return [
        tt.Compose([tt.ToTensor(), tt.Normalize(*stats)]),
        tt.Compose([tt.RandomCrop(28, padding=4, padding_mode='reflect'), 
                        tt.RandomHorizontalFlip(), 
                        tt.ToTensor(), 
                        tt.Normalize(*stats,inplace=True)]),
        tt.Compose([tt.RandomCrop(28, padding=4, padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(p=0.5), 
                         tt.RandomVerticalFlip(p=0.5),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)]),
        tt.Compose([tt.RandomCrop(28, padding=4, padding_mode='reflect'), 
                         tt.ColorJitter(brightness=0.4, 
                             contrast=0.5,
                             saturation=0.4, 
                             hue=0.3),
                         tt.GaussianBlur(kernel_size=(11, 21), sigma=(5, 50)),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)]),
        tt.Compose([tt.RandomCrop(28, padding=4, padding_mode='reflect'),
                    tt.RandomHorizontalFlip(p=0.5), 
                    tt.RandomVerticalFlip(p=0.5),
                 tt.ColorJitter(brightness=0.4, 
                     contrast=0.5,
                     saturation=0.4, 
                     hue=0.3),
                 tt.GaussianBlur(kernel_size=(11, 21), sigma=(5, 50)),
                 tt.ToTensor(), 
                 tt.Normalize(*stats,inplace=True)])]

def get_names():
    return ['no change', 'horizontal flip', '+ vertical flip', 'color jitter and gaussian blur', 'all']

def get_loader(transform):
    basic_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])
    train_ds = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
    train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)

    val_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=False, download=True, transform=basic_tfms)
    valid_dl = DataLoader(val_dataset, batch_size*2, num_workers=3, pin_memory=True)
    train_dl = DeviceDataLoader(train_dl, device)
    valid_dl = DeviceDataLoader(valid_dl, device)
    return train_dl, valid_dl

In [3]:
import torch

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    # elif torch.backends.mps.is_available():
    #     return torch.device("mps")
    else:
        return torch.device("cpu")
    
def clear_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()
    # else:
    #     return torch.device("cpu")
    # appers there's nothing to do here
        
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
    
device = get_default_device()
print(f"running on {device}")

running on cuda


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = self.conv_block(in_channels, 64)
        self.conv2 = self.conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(self.conv_block(128, 128), self.conv_block(128, 128))
        
        self.conv3 = self.conv_block(128, 256, pool=True)
        self.conv4 = self.conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(self.conv_block(512, 512), self.conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(3), 
                                        nn.Flatten(), 
                                        nn.Linear(512, num_classes))

    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  
        loss = F.cross_entropy(out, labels) 
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

    def conv_block(self, in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                nn.BatchNorm2d(out_channels), 
                nn.ReLU(inplace=True)]
        if pool: 
            layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [5]:
model = to_device(ResNet9(1, 10), device)

In [6]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader),
                                                pct_start=0.3)
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [7]:
epochs = 8
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [8]:
history_map = {}
index = 0
for transformation in get_transformations():
    model = to_device(ResNet9(1, 10), device)
    history = []
    train_dl, valid_dl = get_loader(transformation)
    history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, A
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
    history_map[index] = history
    index += 1

Epoch [0], last_lr: 0.00390, train_loss: 0.5059, val_loss: 0.4258, val_acc: 0.8471
Epoch [1], last_lr: 0.00935, train_loss: 0.3815, val_loss: 0.8086, val_acc: 0.7848
Epoch [2], last_lr: 0.00972, train_loss: 0.2519, val_loss: 0.2618, val_acc: 0.9070
Epoch [3], last_lr: 0.00812, train_loss: 0.1869, val_loss: 0.2403, val_acc: 0.9146
Epoch [4], last_lr: 0.00556, train_loss: 0.1463, val_loss: 0.2349, val_acc: 0.9226
Epoch [5], last_lr: 0.00283, train_loss: 0.1025, val_loss: 0.1989, val_acc: 0.9315
Epoch [6], last_lr: 0.00077, train_loss: 0.0568, val_loss: 0.1849, val_acc: 0.9395
Epoch [7], last_lr: 0.00000, train_loss: 0.0297, val_loss: 0.1859, val_acc: 0.9431
Epoch [0], last_lr: 0.00390, train_loss: 0.6833, val_loss: 1.3729, val_acc: 0.6189
Epoch [1], last_lr: 0.00935, train_loss: 0.5266, val_loss: 1.0476, val_acc: 0.7297
Epoch [2], last_lr: 0.00972, train_loss: 0.3955, val_loss: 0.4292, val_acc: 0.8557
Epoch [3], last_lr: 0.00812, train_loss: 0.3066, val_loss: 0.3860, val_acc: 0.8519
Epoc

In [9]:
# comparing different augmentations
import pandas as pd

def compare_final_val_losses():
    # Create a list of final validation losses
    final_val_losses = [history_map[i][-1]['val_loss'] for i in range(len(history_map))]
    final_val_acc = [history_map[i][-1]['val_acc'] for i in range(len(history_map))]

    # Get the transformation names
    names = get_names()

    # Create a pandas DataFrame and display it
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Loss': final_val_losses
    })
    print(df)
    print("*"*10)
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Accuracy': final_val_acc
    })
    print(df)
    

compare_final_val_losses()

                   Transformation  Final Validation Loss
0                       no change               0.185930
1                 horizontal flip               0.190672
2                 + vertical flip               0.218223
3  color jitter and gaussian blur               5.139154
4                             all              12.000578
**********
                   Transformation  Final Validation Accuracy
0                       no change                   0.943130
1                 horizontal flip                   0.931037
2                 + vertical flip                   0.918791
3  color jitter and gaussian blur                   0.277529
4                             all                   0.197108


In [10]:
# redefine to see if we add 1 to the basic if it does better

def get_loader(transform):
    basic_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])
    train_ds_new = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
    train_ds_basic = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=basic_tfms, download=False)
    train_dataset = ConcatDataset([train_ds_basic, train_ds_new])
    train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=3, pin_memory=True)

    val_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=False, download=True, transform=basic_tfms)
    valid_dl = DataLoader(val_dataset, batch_size*2, num_workers=3, pin_memory=True)
    train_dl = DeviceDataLoader(train_dl, device)
    valid_dl = DeviceDataLoader(valid_dl, device)
    return train_dl, valid_dl

In [11]:
history_map = {}
index = 0
for transformation in get_transformations():
    model = to_device(ResNet9(1, 10), device)
    history = []
    train_dl, valid_dl = get_loader(transformation)
    history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
    history_map[index] = history
    index += 1

Epoch [0], last_lr: 0.00393, train_loss: 0.4196, val_loss: 0.3219, val_acc: 0.8850
Epoch [1], last_lr: 0.00935, train_loss: 0.2754, val_loss: 0.3857, val_acc: 0.8776
Epoch [2], last_lr: 0.00972, train_loss: 0.1981, val_loss: 0.3312, val_acc: 0.8863
Epoch [3], last_lr: 0.00812, train_loss: 0.1305, val_loss: 0.2507, val_acc: 0.9127
Epoch [4], last_lr: 0.00556, train_loss: 0.0749, val_loss: 0.3089, val_acc: 0.9104
Epoch [5], last_lr: 0.00283, train_loss: 0.0307, val_loss: 0.2548, val_acc: 0.9345
Epoch [6], last_lr: 0.00077, train_loss: 0.0049, val_loss: 0.2592, val_acc: 0.9418
Epoch [7], last_lr: 0.00000, train_loss: 0.0008, val_loss: 0.2686, val_acc: 0.9425
Epoch [0], last_lr: 0.00393, train_loss: 0.5223, val_loss: 0.5980, val_acc: 0.7715
Epoch [1], last_lr: 0.00935, train_loss: 0.3618, val_loss: 0.4279, val_acc: 0.8542
Epoch [2], last_lr: 0.00972, train_loss: 0.2686, val_loss: 0.4173, val_acc: 0.8607
Epoch [3], last_lr: 0.00812, train_loss: 0.2094, val_loss: 0.2287, val_acc: 0.9157
Epoc

In [12]:
# comparing different augmentations
import pandas as pd

def compare_final_val_losses():
    # Create a list of final validation losses
    final_val_losses = [history_map[i][-1]['val_loss'] for i in range(len(history_map))]
    final_val_acc = [history_map[i][-1]['val_acc'] for i in range(len(history_map))]

    # Get the transformation names
    names = get_names()

    # Create a pandas DataFrame and display it
    df = pd.DataFrame({
        'Basic + Transformation': names,
        'Final Validation Loss': final_val_losses
    })
    print(df)
    print("*"*10)
    df = pd.DataFrame({
        'Basic + Transformation': names,
        'Final Validation Accuracy': final_val_acc
    })
    print(df)
    

compare_final_val_losses()

           Basic + Transformation  Final Validation Loss
0                       no change               0.268636
1                 horizontal flip               0.154221
2                 + vertical flip               0.153414
3  color jitter and gaussian blur               0.171399
4                             all               0.177767
**********
           Basic + Transformation  Final Validation Accuracy
0                       no change                   0.942531
1                 horizontal flip                   0.946953
2                 + vertical flip                   0.947571
3  color jitter and gaussian blur                   0.942187
4                             all                   0.940943


## 